In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm

In [11]:
# RNN model trained to predict the next word in a sentence
class RNNModel(nn.Module):
    def __init__(self, target_sentences):
        super().__init__()
        self.target_sentences = [f"<START> {sentence.strip().lower()} <END>" for sentence in target_sentences]
        self.vocab = list(set(" ".join(self.target_sentences).split(" ")))

        self.hidden_size = 128
        self.embedding_size = 64

        self.last_layer = nn.Linear(self.hidden_size, len(self.vocab))
        self.embedding = nn.Embedding(len(self.vocab), self.embedding_size)
        self.internal_layer = nn.Linear(self.embedding_size + self.hidden_size, self.hidden_size)

        self.loss_fn = nn.CrossEntropyLoss()
        self.optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
    
    def tokenize(self, word):
        return torch.LongTensor([self.vocab.index(word)])

    def logits(self, sentence):

        # Initialize hidden state for sequence
        hidden = torch.zeros(1, self.hidden_size)

        # Process each token in the sequence
        split = sentence.split(" ")
        # Check if the word is in the vocab
        if not all([word in self.vocab for word in split]):
            raise ValueError("Word not in vocab")
        
        # Iterate through each word in the sequence
        for elem in split:
            token = self.tokenize(elem)
            embedding = self.embedding(token)
            input_data = torch.cat((embedding, hidden), dim=1)
            hidden = self.internal_layer(input_data)
            hidden = F.tanh(hidden)
            
        # The last hidden state should go through the last layer
        output = self.last_layer(hidden)

        return output
    
    def get_prob_dist_over_next_word(self, sentence: str):
        dist = self.forward(sentence)[0]
        return {self.vocab[i]: f"{dist[i].item():.2f}" for i in range(len(dist))}

    def forward(self, sentence):
        return F.softmax(self.logits(sentence), dim=1)
    
    def predict(self, sentence):
        result = self.vocab[torch.argmax(self.forward(sentence.strip()))]
        return result
    
    def complete_sentence(self, sentence, max_len=100):
        result = sentence.strip()
        new_token = ""
        while not new_token == "<END>":
            new_token = self.predict(result)
            if len(result.split(" ")) >= max_len:
                break
            result += " " + new_token
        return result

    def train(self, num_epochs=10):
        for epoch in tqdm(range(num_epochs), unit="epoch"):
            for sentence in self.target_sentences:
                split_sentence = sentence.split(" ")
                for i in range(len(split_sentence) - 1):
                    self.optimizer.zero_grad()
                    input_data = split_sentence[:i+1]
                    target = split_sentence[i+1]
                    output = self.forward(" ".join(input_data))
                    loss = self.loss_fn(output, self.tokenize(target))
                    loss.backward()
                    self.optimizer.step()
        
                        

Før vi trener modellen sjekker vi at den **ikke** klarer å skrive setninger som gir mening

In [12]:
target_sentences = ["dette er et kodekurs i maskinlæring", "kodekurset er gøy fordi man lærer mye"]
rnn = RNNModel(target_sentences)
rnn.complete_sentence("<START> ")

'<START> dette lærer kodekurset <START> er <START> er lærer er kodekurset <START> dette lærer er lærer er kodekurset <START> dette lærer er lærer er kodekurset <START> dette lærer er lærer er kodekurset <START> dette lærer er lærer er kodekurset <START> dette lærer er lærer er kodekurset <START> dette lærer er lærer er kodekurset <START> dette lærer er lærer er kodekurset <START> dette lærer er lærer er kodekurset <START> dette lærer er lærer er kodekurset <START> dette lærer er lærer er kodekurset <START> dette lærer er lærer er kodekurset <START> dette lærer er lærer er kodekurset <START> dette lærer er lærer'

In [17]:
rnn.train(num_epochs=10)

100%|██████████| 10/10 [00:00<00:00, 32.19epoch/s]


In [18]:
rnn.complete_sentence("<START> dette")

'<START> dette er et kodekurs i maskinlæring <END>'

In [19]:
rnn.get_prob_dist_over_next_word("<START> dette er et")

{'er': '0.00',
 'kodekurs': '0.97',
 'i': '0.00',
 'et': '0.00',
 'kodekurset': '0.00',
 'maskinlæring': '0.00',
 'lærer': '0.00',
 'gøy': '0.00',
 '<START>': '0.00',
 '<END>': '0.00',
 'dette': '0.00',
 'mye': '0.00',
 'man': '0.00',
 'fordi': '0.01'}

In [24]:
sentences  = [
    "Katten sov rolig under det store treet.",
    "En rolig dag ved sjøen kan være veldig avslappende.",
    "Han leste en bok under et skyggefullt tre.",
    "Solen skinte sterkt, og fuglene sang vakkert.",
    "En rask løpetur om morgenen gjør meg energisk for dagen.",
    "Hun malte et bilde av et fargerikt tre.",
    "Å lage mat sammen kan være en morsom aktivitet.",
    "Vannet i innsjøen var krystallklart og rolig.",
    "En bok om eventyr kan være veldig spennende.",
    "Morgensolen varmer og gir energi til en ny dag.",
    "Han spilte gitar under stjernene og sang rolig.",
    "Å gå på tur i skogen kan være både fredelig og energigivende.",
    "Boken på bordet er full av interessante historier.",
    "Fuglene bygger reir i det høye treet.",
    "Å se på stjernene om natten er en fredelig opplevelse.",
    "Å spise sunn mat gir god energi for kroppen.",
    "En rolig spasertur i parken kan være veldig hyggelig.",
    "Å lese bøker under et tre er en av mine favorittaktiviteter.",
    "Morgendugg på bladene skinte som små diamanter.",
    "Trening hver morgen holder meg aktiv og energisk."
]
rnn = RNNModel(sentences)
rnn.complete_sentence("<START>")


'<START> innsjøen sov morsom rolig morgen kan bok rask bordet stjernene tre. gir gjør under tre. gir et mat rolig bilde full katten gå et morsom bygger sunn aktiv opplevelse. sjøen mat morsom rolig morgen kan bok rask bordet stjernene tre. gir gjør under tre. gir et mat rolig bilde full katten gå et morsom bygger sunn aktiv opplevelse. sjøen mat morsom rolig morgen kan bok rask bordet stjernene tre. gir gjør under tre. gir et mat rolig bilde full katten gå et morsom bygger sunn aktiv opplevelse. sjøen mat morsom rolig morgen kan bok rask bordet stjernene tre. gir'

In [27]:
rnn.train(num_epochs=50)

100%|██████████| 50/50 [00:25<00:00,  1.99epoch/s]


In [28]:
rnn.complete_sentence("<START> ")

'<START> å se på stjernene om natten er en fredelig opplevelse. <END>'

In [144]:
class LSTMModel(nn.Module):
    def __init__(self, target_sentences):
        super().__init__()
        self.target_sentences = [f"<START> {sentence.strip().lower()} <END>" for sentence in target_sentences]
        self.vocab = list(set(" ".join(self.target_sentences).split(" ")))

        self.hidden_size = 128
        self.embedding_size = 64

        self.lstm = nn.LSTM(self.embedding_size, self.hidden_size, batch_first=True, bidirectional=False)
        self.last_layer = nn.Linear(self.hidden_size, len(self.vocab))
        self.embedding = nn.Embedding(len(self.vocab), self.embedding_size)

        self.loss_fn = nn.CrossEntropyLoss()
        self.optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
    
    def tokenize(self, word):
        return torch.LongTensor([self.vocab.index(word)])

    def logits(self, sentence):
        # Initialize hidden and cell state for sequence
        hidden = torch.zeros(1, 1, self.hidden_size)
        cell = torch.zeros(1, 1, self.hidden_size)

        # Process each token in the sequence
        split = sentence.split(" ")
        if not all([word in self.vocab for word in split]):
            raise ValueError("Word not in vocab")
        
        # Embedding layer
        tokenized = [self.tokenize(word) for word in split]
        embedded = self.embedding(torch.stack(tokenized).squeeze(1))    
        embedded = embedded.view(1, len(embedded), -1)

        # LSTM layer
        lstm_out, _ = self.lstm(embedded, (hidden, cell))
        output = self.last_layer(lstm_out[:, -1])
        return output

    def get_prob_dist_over_next_word(self, sentence: str):
        dist = self.forward(sentence)[0]
        return {self.vocab[i]: f"{dist[i].item():.2f}" for i in range(len(dist))}

    def forward(self, sentence):
        return F.softmax(self.logits(sentence), dim=1)
    
    def predict(self, sentence):
        result = self.vocab[torch.argmax(self.forward(sentence))]
        return result
    
    def complete_sentence(self, sentence, max_len=100):
        result = sentence.strip()
        new_token = ""
        while not new_token == "<END>":
            new_token = self.predict(result)
            if len(result.split(" ")) >= max_len:
                break
            result += " " + new_token
        return result

    def train(self, num_epochs=10):
        for epoch in tqdm(range(num_epochs), unit="epoch"):
            for sentence in self.target_sentences:
                split_sentence = sentence.split(" ")
                for i in range(len(split_sentence) - 1):
                    self.optimizer.zero_grad()
                    input_data = split_sentence[:i+1]
                    target = split_sentence[i+1]
                    output = self.forward(" ".join(input_data))
                    loss = self.loss_fn(output, self.tokenize(target))
                    loss.backward()
                    self.optimizer.step()
        

In [145]:
lstm = LSTMModel(sentences)
lstm.complete_sentence("<START> ")

'<START> mens fylte blomstret blomstret blomstret skyggefulle mens fylte blomstret blomstret skyggefulle mens fylte blomstret blomstret skyggefulle mens fylte blomstret blomstret skyggefulle mens fylte blomstret blomstret skyggefulle mens fylte blomstret blomstret skyggefulle mens fylte blomstret blomstret skyggefulle mens fylte blomstret blomstret skyggefulle mens fylte blomstret blomstret skyggefulle mens fylte blomstret blomstret skyggefulle mens fylte blomstret blomstret skyggefulle mens fylte blomstret blomstret skyggefulle mens fylte blomstret blomstret skyggefulle mens fylte blomstret blomstret skyggefulle mens fylte blomstret blomstret skyggefulle mens fylte blomstret blomstret skyggefulle mens fylte blomstret blomstret skyggefulle mens fylte blomstret blomstret skyggefulle mens fylte blomstret blomstret skyggefulle mens fylte blomstret'

In [151]:
lstm.train(num_epochs=100)

100%|██████████| 100/100 [00:15<00:00,  6.36epoch/s]


In [153]:
lstm.complete_sentence("<START> parken")

'<START> parken i parken sang fuglene mens folk gikk på stier. <END>'